In [1]:
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import get_embeddings
import os
import openai
import json
from Keys import openai_keys
import re
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn import tree
from sklearn.cluster import KMeans
from ast import literal_eval
import numpy as np
import random
import joblib
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow import keras
from keras import layers
import plotly.express as px
import plotly.graph_objects as go
openai.organization = openai_keys['organization']
openai.api_key = openai_keys['api_key']
embedding_model = "text-embedding-ada-002"
def leaky_relu(x):
    return tf.where(x > 0, x, 0.01 * x)

def embed(X, embedding_model="text-embedding-ada-002"):
    curB = 0
    Y = [0] * len(X)
    while 2048 * curB < len(X):
        Y_loc = get_embeddings(X[2048 * curB: min(2048 * (curB + 1), len(X))], embedding_model)
        for i in range(len(Y_loc)):
            Y[2048 * curB + i] = Y_loc[i]
        curB += 1
    return Y


In [ ]:
with open('Datasets/ForParsing/Clothing_and_Jewelry.json', 'r') as json_file:
    data = json.load(json_file)


In [23]:
with open('Datasets/ForParsing/CDs_and_Vinyl_500k.json', 'r') as json_file:
    data = json.load(json_file)
random.shuffle(data)
df = pd.DataFrame(data)[['reviewText', 'overall', 'summary']]
negatives = df[df['overall'] <= 2][:20000]
positives = df[df['overall'] >= 4][:20000]
neutrals = df[df['overall'] == 3][:10000]
df = pd.concat([negatives, neutrals, positives], ignore_index=True)
df = df[df['reviewText'].str.len() <= 15000]
# df["embedding"] = df.apply(lambda row: get_embedding(str(row['summary']) + " : " + str(row['reviewText']), engine=embedding_model), axis=1)
df["embedding"] = embed(list(df['reviewText'].astype(str)))
df.to_csv("Datasets/Embeddings/CDs_and_Vinyl_Embeddings_50k.csv")
# display(df['overall'].value_counts())

In [19]:
df = pd.read_csv('Datasets/ForParsing/chatGPTSentiment.csv')
df.dropna(axis=0)
negatives = df[df['labels'] == 'bad'][:3000]
positives = df[df['labels'] == 'good'][:3000]
neutrals = df[df['labels'] == 'neutral'][:4000]
df = pd.concat([negatives, neutrals, positives], ignore_index=True)
df = df[df['tweets'].str.len() <= 15000]
df["embedding"] = embed(list(df['tweets'].astype(str)))
df.to_csv("Datasets/Embeddings/chatGPTSentiment_embeddings_10k.csv")

In [18]:
df = pd.read_csv('Datasets/ForParsing/financialData.csv')
df = df[df['text'].str.len() <= 15000]
df["embedding"] = embed(list(df['text'].astype(str)), embedding_model)
df.to_csv("Datasets/Embeddings/financeData1_embeddings.csv")

In [32]:
df = pd.read_csv('Datasets/ForParsing/AirlineSentiment.csv')
df = df[df['airline_sentiment_confidence'] > 0.9]
df = df[['airline_sentiment', 'text']]
df = df[df['text'].str.len() <= 15000]
df['text'] = df['text'].apply(lambda x: x.replace("@VirginAmerica", ""))
df["embedding"] = embed(list(df['text'].astype(str)), embedding_model)
df.to_csv("Datasets/Embeddings/airline_twitter_embeddings.csv")

In [35]:
df = pd.read_csv('Datasets/ForParsing/SentimentGOP.csv')
df = df[df['sentiment_confidence'] > 0.9]
df = df[['sentiment', 'text']]
df = df[df['text'].str.len() <= 15000]
df["embedding"] = embed(list(df['text'].astype(str)), embedding_model)
df.to_csv("Datasets/Embeddings/gop_twitter_embeddings.csv")

In [48]:
filepaths = ['Datasets/ForParsing/sentiment labelled sentences/yelp_labelled.txt',
                 'Datasets/ForParsing/sentiment labelled sentences/amazon_cells_labelled.txt',
                 'Datasets/ForParsing/sentiment labelled sentences/imdb_labelled.txt']

df_list = []
for filepath in filepaths:
    df = pd.read_csv(filepath, names=['sentence', 'sentiment'], sep='\t')
    df_list.append(df)
df = pd.concat(df_list)
df["embedding"] = embed(list(df['sentence'].astype(str)), embedding_model)
df.to_csv("Datasets/Embeddings/HQ_sentences_embeddings.csv")

In [15]:
with open('Datasets/ForParsing/negative-words.txt', 'r', encoding='utf-8', errors='replace') as file:
    lines = file.readlines()

series = pd.read_csv('Datasets/ForParsing/positive-words.txt', header=None, delimiter='¬', squeeze=True)
series2 = pd.Series(lines)
series2 = series2.str.replace('\n', '')
df1 = pd.DataFrame(series)
df1['sentiment'] = 'positive'
df2 = pd.DataFrame(series2)
df2['sentiment'] = 'negative'
df = pd.concat([df1, df2])
df.rename(columns={0: 'word'}, inplace=True)
df['embedding'] = embed(list(df['word']), embedding_model)
df.to_csv("Datasets/Embeddings/keywords_embeddings.csv")

C:\Users\Admin\AppData\Local\Temp\ipykernel_11276\3345169400.py:4: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  series = pd.read_csv('Datasets/ForParsing/positive-words.txt', header=None, delimiter='¬', squeeze=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11276\3345169400.py:4: FutureWarning: The squeeze argument has been deprecated and will be removed in a future version. Append .squeeze("columns") to the call to squeeze.


  series = pd.read_csv('Datasets/ForParsing/positive-words.txt', header=None, delimiter='¬', squeeze=True)


In [2]:
df = pd.read_csv('Datasets/Embeddings/trip_advisor_embeddings_full.csv')
df2 = pd.read_csv('Datasets/Embeddings/CDs_and_Vinyl_Embeddings_50k.csv')
df3 = pd.read_csv('Datasets/Embeddings/chatGPTSentiment_embeddings_10k.csv')
df4 = pd.read_csv('Datasets/Embeddings/financeData1_embeddings.csv')
df5 = pd.read_csv('Datasets/Embeddings/financeData2_embeddings.csv')
df6 = pd.read_csv("Datasets/Embeddings/keywords_embeddings.csv")
df7 = pd.read_csv("Datasets/Embeddings/airline_twitter_embeddings.csv")
df8 = pd.read_csv("Datasets/Embeddings/gop_twitter_embeddings.csv")
df9 = pd.read_csv("Datasets/Embeddings/HQ_sentences_embeddings.csv")
Y_df = pd.concat([df.Rating.apply(lambda x: 0 if x <= 3 else 2), df2.overall.apply(lambda x: 0 if x <= 3 else 2), df3.labels.apply(lambda x: 0 if x == "bad" else (1 if "neutral" else 2)), df4.sentiment.apply(lambda x: 0 if x == "negative" else (1 if "neutral" else 2)), df5.Sentiment.apply(lambda x: 0 if x == "negative" else (1 if "neutral" else 2)), df6.sentiment.apply(lambda x: 0 if x == "negative" else 2), df7.airline_sentiment.apply(lambda x: 0 if x == "negative" else (1 if "neutral" else 2)), df8.sentiment.apply(lambda x: 0 if x == "Negative" else (1 if "Neutral" else 2)), df9.sentiment.apply(lambda x: 0 if x == 0 else 2)])
X_df = pd.concat([df.embedding, df2.embedding, df3.embedding, df4.embedding, df5.embedding, df6.embedding, df7.embedding, df8.embedding, df9.embedding])
X_df = X_df.apply(literal_eval).apply(np.array)  # convert string to array

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    list(X_df.values), list(Y_df.values), test_size=0.05, random_state=69
)

In [20]:
# train random forest classifier
clf = RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000)

In [68]:
# joblib.dump(clf, 'models/embeddingsForest_4.pkl')

clf = joblib.load('models/embeddingsForest_4.pkl')

In [22]:
preds = clf.predict(X_test)
probas = clf.predict_proba(X_test)

report = classification_report(y_test, preds)
print(report)

              precision    recall  f1-score   support

           0       0.88      0.86      0.87      2231
           1       0.85      0.93      0.89       842
           2       0.90      0.88      0.89      1825

    accuracy                           0.88      4898
   macro avg       0.88      0.89      0.88      4898
weighted avg       0.88      0.88      0.88      4898



In [67]:
#inference
clf.predict(np.array(get_embedding("This movie has some of the worst posters in years. This is such amateur composition and photoshopping.", engine=embedding_model)).reshape(1, -1))

array([0], dtype=int64)

In [4]:
X_tensor_train = tf.convert_to_tensor(np.array(X_train), dtype=tf.float32)
X_tensor_test = tf.convert_to_tensor(np.array(X_test), dtype=tf.float32)

Y_tensor_train = tf.convert_to_tensor(pd.get_dummies(y_train).to_numpy())
Y_tensor_test = tf.convert_to_tensor(pd.get_dummies(y_test).to_numpy())



In [8]:

hyperOptimizer = tf.keras.optimizers.Adam(learning_rate=0.0001, weight_decay=0, amsgrad=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='models/temp/NNmodel.h5',
                             monitor='val_categorical_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max')
NNmodel = tf.keras.Sequential([tf.keras.layers.Dense(200, activation=tf.keras.activations.relu), tf.keras.layers.Dropout(0.3),  tf.keras.layers.Dense(3)])
NNmodel.compile(optimizer=hyperOptimizer, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=(tf.keras.metrics.CategoricalAccuracy()))
history = NNmodel.fit(X_tensor_train, Y_tensor_train, batch_size=256, epochs=500, validation_data=(X_tensor_test, Y_tensor_test), callbacks=[checkpoint])
NNmodel.evaluate(X_tensor_test, Y_tensor_test, verbose=1)


Epoch 1/500
429/433 [============================>.] - ETA: 0s - loss: 0.6597 - categorical_accuracy: 0.7418
Epoch 1: val_categorical_accuracy improved from -inf to 0.85962, saving model to models/temp\NNmodel.h5
433/433 [==============================] - 5s 7ms/step - loss: 0.6577 - categorical_accuracy: 0.7428 - val_loss: 0.4150 - val_categorical_accuracy: 0.8596
Epoch 2/500
432/433 [============================>.] - ETA: 0s - loss: 0.3581 - categorical_accuracy: 0.8685
Epoch 2: val_categorical_accuracy improved from 0.85962 to 0.87198, saving model to models/temp\NNmodel.h5
433/433 [==============================] - 3s 7ms/step - loss: 0.3581 - categorical_accuracy: 0.8685 - val_loss: 0.3283 - val_categorical_accuracy: 0.8720
Epoch 3/500
432/433 [============================>.] - ETA: 0s - loss: 0.3108 - categorical_accuracy: 0.8778
Epoch 3: val_categorical_accuracy improved from 0.87198 to 0.87524, saving model to models/temp\NNmodel.h5
433/433 [==============================] - 3s

[0.2541351914405823, 0.9078428149223328]

In [48]:
hyperOptimizer = tf.keras.optimizers.Adam(learning_rate=0.001, amsgrad=True)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='models/temp/ResNetmodel.h5',
                             monitor='val_categorical_accuracy',
                             verbose=0,
                             save_best_only=True,
                             mode='max')


def generateModel():
    inputs = keras.Input(shape=(1536, ), name="emb")
    x = layers.Dense(100, activation=keras.activations.swish)(inputs)
    # x = layers.Dropout(0.1)(x)
    x = layers.Activation(keras.activations.swish)(layers.Dense(100)(x) + layers.Dense(100)(inputs))
    # x = layers.Dropout(0.1)(x)
    # x = keras.activations.relu(layers.Dense(100)(x) + baseModel_out)

    outputs = layers.Dense(3)(x)
    return keras.Model(inputs, outputs)

ResNetmodel = generateModel()
# ResNetmodel.summary()
ResNetmodel.compile(optimizer=hyperOptimizer, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=(tf.keras.metrics.CategoricalAccuracy()))
history = ResNetmodel.fit(X_tensor_train, Y_tensor_train, batch_size=512, epochs=25, validation_data=(X_tensor_test, Y_tensor_test), callbacks=[checkpoint], verbose=1)
ResNetmodel.evaluate(X_tensor_test, Y_tensor_test, verbose=0)

Epoch 1/25
217/217 [==============================] - 9s 20ms/step - loss: 0.3713 - categorical_accuracy: 0.8442 - val_loss: 0.2810 - val_categorical_accuracy: 0.8862
Epoch 2/25
217/217 [==============================] - 5s 22ms/step - loss: 0.2606 - categorical_accuracy: 0.8920 - val_loss: 0.2666 - val_categorical_accuracy: 0.8903
Epoch 3/25
217/217 [==============================] - 5s 22ms/step - loss: 0.2478 - categorical_accuracy: 0.8973 - val_loss: 0.2602 - val_categorical_accuracy: 0.8909
Epoch 4/25
217/217 [==============================] - 4s 17ms/step - loss: 0.2400 - categorical_accuracy: 0.9004 - val_loss: 0.2639 - val_categorical_accuracy: 0.8900
Epoch 5/25
217/217 [==============================] - 4s 16ms/step - loss: 0.2341 - categorical_accuracy: 0.9034 - val_loss: 0.2549 - val_categorical_accuracy: 0.8945
Epoch 6/25
217/217 [==============================] - 4s 17ms/step - loss: 0.2295 - categorical_accuracy: 0.9065 - val_loss: 0.2496 - val_categorical_accuracy: 0.896

[0.24140934646129608, 0.9026943445205688]

In [142]:
hyperOptimizer = tf.keras.optimizers.Adam(learning_rate=0.01, weight_decay=0)
CNNmodel = tf.keras.Sequential([tf.keras.layers.Conv1D(filters=25, kernel_size=16, input_shape=(1536, 1)), tf.keras.layers.MaxPool1D(pool_size=2, strides=2), tf.keras.layers.Conv1D(filters=5, kernel_size=8), tf.keras.layers.AvgPool1D(pool_size=2, strides=2), tf.keras.layers.Conv1D(filters=1, kernel_size=4), tf.keras.layers.Flatten(), tf.keras.layers.Dense(3)])
# print(CNNmodel.summary())
CNNmodel.compile(optimizer=hyperOptimizer, loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=(tf.keras.metrics.CategoricalAccuracy()))
history = CNNmodel.fit(X_tensor_train, Y_tensor_train, batch_size=256, epochs=10, validation_data=(X_tensor_test, Y_tensor_test))

CNNmodel.evaluate(X_tensor_test, Y_tensor_test, verbose=2)

Epoch 1/10
433/433 [==============================] - 55s 123ms/step - loss: 0.3593 - categorical_accuracy: 0.8487 - val_loss: 0.3182 - val_categorical_accuracy: 0.8711
Epoch 2/10
433/433 [==============================] - 66s 153ms/step - loss: 0.2996 - categorical_accuracy: 0.8784 - val_loss: 0.3129 - val_categorical_accuracy: 0.8764
Epoch 3/10
433/433 [==============================] - 66s 152ms/step - loss: 0.2898 - categorical_accuracy: 0.8821 - val_loss: 0.3077 - val_categorical_accuracy: 0.8773
Epoch 4/10
433/433 [==============================] - 61s 140ms/step - loss: 0.2865 - categorical_accuracy: 0.8839 - val_loss: 0.3017 - val_categorical_accuracy: 0.8785
Epoch 5/10
433/433 [==============================] - 63s 145ms/step - loss: 0.2801 - categorical_accuracy: 0.8855 - val_loss: 0.3041 - val_categorical_accuracy: 0.8721
Epoch 6/10
433/433 [==============================] - 59s 137ms/step - loss: 0.2800 - categorical_accuracy: 0.8867 - val_loss: 0.2948 - val_categorical_acc

[0.29252496361732483, 0.878839910030365]

In [155]:
NNmodel_inf = tf.keras.Sequential([NNmodel, tf.keras.layers.Softmax()])
report = classification_report(np.argmax(Y_tensor_test, axis=1), np.argmax(NNmodel_inf.predict(X_tensor_test), axis=1))
print(report)

183/183 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

           0       0.92      0.90      0.91      2883
           1       0.89      0.92      0.91      1044
           2       0.90      0.92      0.91      1900

    accuracy                           0.91      5827
   macro avg       0.91      0.91      0.91      5827
weighted avg       0.91      0.91      0.91      5827



In [167]:
# NNmodel.save("models/embeddingsNN")
NNmodel = tf.keras.models.load_model("models/temp/NNmodel.h5")

In [45]:
# Plot the training history
fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['loss'], mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(y=history.history['val_loss'], mode='lines', name='Validation Loss'))
fig.update_layout(title='Training Loss', xaxis_title='Epoch', yaxis_title='Loss')
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(y=history.history['categorical_accuracy'], mode='lines', name='Train Accuracy'))
fig.add_trace(go.Scatter(y=history.history['val_categorical_accuracy'], mode='lines', name='Validation Accuracy'))
fig.update_layout(title='Training Accuracy', xaxis_title='Epoch', yaxis_title='Accuracy')
fig.show()

In [168]:
NNmodel_inf = tf.keras.Sequential([NNmodel, tf.keras.layers.Softmax()])
validate = np.hstack((np.round(NNmodel_inf.predict([X_tensor_test[:100, :]]), 2), np.array(y_test[:100]).reshape((-1, 1))))

4/4 [==============================] - 0s 3ms/step


In [71]:
validate_tree = np.hstack((clf.predict_proba(X_test[:100]), np.array(y_test[:100]).reshape((-1, 1))))

In [169]:
print('ANN Accuracy: ' + str(np.count_nonzero(np.argmax(NNmodel_inf.predict([X_tensor_test]), axis=1) == y_test)/len(y_test)))
print('RF Accuracy: ' + str(np.count_nonzero(clf.predict(X_test) == y_test)/len(y_test)))

183/183 [==============================] - 0s 2ms/step
ANN Accuracy: 0.9104170241977003
RF Accuracy: 0.9028659687660889


In [86]:
clf.predict(X_test)

array([1, 1, 0, ..., 2, 2, 2], dtype=int64)